МОЙ ВКЛАД

https://www.banki.ru/products/deposits/deposit/20217/?rate_id=2712310&amount=0

ДЛЯ ПРОВЕРКИ

https://www.banki.ru/products/deposits/deposit/20147/?rate_id=2707353&amount=0
https://www.banki.ru/products/deposits/deposit/14907/

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

url = 'https://www.banki.ru/products/deposits/deposit/14907/'
r = requests.get(url)
with open('text.html', 'w') as output_file:
    output_file.write(r.text)
    
result = pd.DataFrame()

soup = BeautifulSoup(r.text) #Отправляем полученную страницу в библиотеку для парсинга

# Нашли все пункты раздела "Параметры вклада"
parameters_contribution=soup.find_all('div', {'class': 'border-bottom-colorMinorGrayLighten deposit-info-params-item flexbox padding-vert-medium'}) 


name_v=soup.h1.text.strip()

print(name_v)

Вклад СмартВклад (с повышенной ставкой) Тинькофф Банк


In [2]:
#  - Возвращает возможность снятия

def get_withdrawal_possibility(soup):
    snyat = soup.find("dt",text="Частичное снятие").findNext("dd")
    snyat = str(snyat)
    pattern = re.compile('[а-яА-ЯёЁ/ 0-9,]+[;:]?')
    spisok_rab = pattern.findall(snyat)
    str_rab = ''.join(spisok_rab)
    not_usl = str_rab.lower()
    if str_rab == 'возможно' or str_rab == 'невозможно':
        return not_usl.title()
    else:
        s_usl = spisok_rab[0]
        s_usl = re.sub('[;:]', ' ', s_usl)
        s_usl = s_usl.rstrip()
        return s_usl
    
get_withdrawal_possibility(soup)   


'возможно, но есть ограничения'

In [3]:
#  - Возвращает возможность пролонгации (возможна/невозможна/нет данных)

def get_prolongation_possibility(soup):
    try:
        prolong = soup.find("dt",text="Автопролонгация").findNext("dd").string.strip()
        prolong = str(prolong)
        prolong = prolong.lower()
        if prolong == 'возможна' or prolong == 'невозможна':
            return prolong.title()
        else:
            return 'Нет данных'
    except AttributeError:
        return 'Нет данных'        

# try-except - если такого пункта на сайте вообще нет, то будет делаться это

get_prolongation_possibility(soup)

'Возможна'

In [4]:
#  - Возвращает условия снятия (если они есть) в виде списка, в случае отсутствия - None

def get_withdrawal_conditions(soup):
    snyat = soup.find("dt",text="Частичное снятие").findNext("dd")
    snyat = str(snyat)
    pattern = re.compile('[а-яА-ЯёЁ/ 0-9,]+[;:]?')
    spisok_usl = pattern.findall(snyat)
    stroka_usl = ''.join(spisok_usl)
    stroka_usl = re.sub('[/]', ' ', stroka_usl)
    spisok_usl = stroka_usl.split(":")
    if spisok_usl[0] == 'возможно, но есть ограничения':
        stroka_usl = spisok_usl[1]
        del spisok_usl[0]
    spisok_usl = stroka_usl.split(";")
    for t in range(len(spisok_usl)):
        rab_str = spisok_usl[t]
        rab_str = rab_str.rstrip()
        rab_str = rab_str.strip()
        rab_str = re.sub('[ ][ ]+[a-zA-Zа-яА-Я0-9;:][ ][ ]+', ' ', rab_str)
        spisok_usl[t] = rab_str
    if len(spisok_usl) == 1:
        return None
    else:
        return spisok_usl
get_withdrawal_conditions(soup)

['минимальная сумма снятия 15 000 рублей, 100 долларов евро',
 'допускается не ранее чем через 60 дней после открытия вклада',
 'проценты будут удерживаться пропорционально сумме изъятия и пересчитываться исходя из ставки 0,01 годовых на сумму изъятия',
 'на остаток средств на счете вклада после частичного изъятия продолжают начисляться проценты согласно действующей ставке по вкладу']

In [11]:
#  - Возвращает условия пополнения (если они есть) в виде списка, в случае отсутствия - None

def get_replenishment_conditions(soup):
    popoln = soup.find("dt",text="Пополнение").findNext("dd")
    popoln = str(popoln)
    pattern = re.compile('[а-яА-ЯёЁ/ 0-9,]+[;:]?')
    spisok_usl_pop = pattern.findall(popoln)
    stroka_usl_pop = ''.join(spisok_usl_pop)
    stroka_usl_pop = re.sub('[/]', ' ', stroka_usl_pop)
    spisok_usl_pop = stroka_usl_pop.split(":")
    if spisok_usl_pop[0] == 'возможно, но есть ограничения':
        stroka_usl_pop = spisok_usl_pop[1]
        del spisok_usl_pop[0]
    spisok_usl_pop2 = stroka_usl_pop.split(";")
    for t in range(len(spisok_usl_pop2)):
        rab_str_u_p = spisok_usl_pop2[t]
        rab_str_u_p = rab_str_u_p.rstrip()
        rab_str_u_p = rab_str_u_p.strip()
        rab_str_u_p = re.sub('[ ][ ]+[a-zA-Zа-яА-Я0-9;:][ ][ ]+', ' ', rab_str_u_p)
        spisok_usl_pop2[t] = rab_str_u_p
    if len(spisok_usl_pop2) == 0:
        return None
    else:
        return spisok_usl_pop2
    
get_replenishment_conditions(soup)   

['срок внесения пополнения в первые 30 дней с момента открытия вклада']

In [7]:
#  - Возвращает список с особыми условиями 

def get_custom_conditions(soup):
    try:
        osob_usl = soup.find("dt",text="Особые условия").findNext("dd")
        osob_usl = str(osob_usl)
        pattern = re.compile('[а-яА-ЯёЁ/ 0-9,]+[;:]?')
        spisok_osob_usl = pattern.findall(osob_usl)
        stroka_osob_usl = ''.join(spisok_osob_usl)
        stroka_osob_usl = re.sub('[/]', ' ', stroka_osob_usl)
        spisok_osob_usl = stroka_osob_usl.split(";")
        for t in range(len(spisok_osob_usl)):
            rab_str_o_u = spisok_osob_usl[t]
            rab_str_o_u = rab_str_o_u.rstrip()
            rab_str_o_u = rab_str_o_u.strip()
            rab_str_o_u = re.sub('[ ][ ]+[a-zA-Zа-яА-Я0-9;:][ ][ ]+', ' ', rab_str_o_u)
            spisok_osob_usl[t] = rab_str_o_u
        if len(spisok_osob_usl) == 1:
            return None
        else:
            return spisok_osob_usl
    except AttributeError:
        return None
    
    
    

get_custom_conditions(soup)

['вклад открывается дистанционно',
 'комиссия за открытие вклада в офисе банка  1 000 рублей',
 'для вкладов, открытых дистанционно, предусмотрена возможность перевода моновалютного вклада в мультивалютный по курсу банка без потери накопленных процентов',
 'минимальная сумма конвертации 1 000 рублей, 10 долларов евро']